In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import librosa, librosa.display
#from pytorchtools import EarlyStopping


if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  2.1.0+cu121  Device :  cuda


In [3]:
#3. 하이퍼 파라미터
BATCH_SIZE =  64 #한 배치당 32개 이미지데이터
EPOCHS = 50# 전체 데이터 셋을 40번 반복

In [6]:
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

train_data_dir='/content/drive/MyDrive/food/kfood_health_train'
# Create the ImageFolder dataset
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(-25, 25)),
    transforms.ColorJitter(brightness=0.6, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
dataset = ImageFolder(root=train_data_dir, transform=transform)

# Define the sizes for the training and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # Remaining for validation

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
validation_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [7]:
test_data_dir = '/content/drive/MyDrive/food/kfood_health_val'
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the same size used in training
    transforms.ToTensor(),  # Convert to a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize using the same mean and standard deviation as training
])
test_dataset = ImageFolder(root=test_data_dir, transform=transform_test)

test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

In [13]:
import torch
import torchvision.models as models

# ResNet-18 모델을 초기화합니다.
model2 = models.resnet18()

# 불러올 체크포인트 파일 경로를 지정합니다.
checkpoint_path = '/content/drive/MyDrive/food/부트캠프 미션1 50 parameter.pt'

# 저장된 모델 가중치를 불러옵니다.
checkpoint = torch.load(checkpoint_path)

# 모델의 가중치를 불러온 체크포인트로 업데이트합니다.
model2.load_state_dict(checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', 'layer2.0.bn2.running_mean', 'layer2.0.bn2.running_var', 'layer2.0.downsample.0.weight', 'layer2.0.downsample.1.weight', 'layer2.0.downsample.1.bias', 'layer2.0.downsample.1.running_mean',

In [14]:
# 모델
# pretrained

classes=['가리비','갈비찜','고등어','김치국','낚지볶음','돼지갈비찜','된장찌개','떡국','모듬초밥','배추찌개','부대찌개','순대','오리로스구이']
def model_initialize():
    model = model2
    model.ftrs = model.fc.in_features # in_features : fully connected의 입력수.
    num_ftrs = model.fc.in_features

    # 1000 -> 10 로 fc를 수정. class가 10개니까
    model.fc = nn.Sequential(nn.Linear(num_ftrs, len(classes)),)

    model = model.cuda()
    return model
model=model_initialize()

In [15]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.00015)
scheduler = StepLR(optimizer, step_size=12, gamma=0.5)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx,(image,label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        train_loss += loss.item()
        prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
        correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.
    train_loss/=len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss,train_accuracy

In [11]:
#9. 학습 진행하며, validation 데이터로 모델 성능확인
def evaluate(model,valid_loader):
    model.eval()
    misclassified_images = []
    valid_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label in valid_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            valid_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
                        # Check for misclassified images
            for i in range(label.size(0)):
                if prediction[i] != label[i]:
                    misclassified_images.append((image[i], prediction[i], label[i]))

            #true.false값을 sum해줌. item
        valid_loss /= len(valid_loader.dataset)
        valid_accuracy = 100. * correct / len(valid_loader.dataset)
        return valid_loss,valid_accuracy,misclassified_images

In [16]:
patience = 5  # Define your patience value
best_valid_acc = 0  # Initialize the best validation accuracy

for Epoch in range(1, EPOCHS + 1):
    train_loss, train_accuracy = train(model, train_loader, optimizer, log_interval=31)
    valid_loss, valid_accuracy,_ = evaluate(model, validation_loader)

    scheduler.step()

    print("\n[EPOCH:{}]\t Train Loss:{:.4f}\t Train Acc:{:.2f} %  | \tValid Loss:{:.4f} \tValid Acc: {:.2f} %\n".
        format(Epoch, train_loss, train_accuracy, valid_loss, valid_accuracy))

    # Check if the validation accuracy improved
    if valid_accuracy > best_valid_acc:
        best_valid_acc = valid_accuracy
        # Reset the patience count because the validation accuracy improved
        patience_counter = 0
    else:
        patience_counter += 1


[EPOCH:1]	 Train Loss:0.0297	 Train Acc:35.97 %  | 	Valid Loss:0.0252 	Valid Acc: 47.64 %


[EPOCH:2]	 Train Loss:0.0216	 Train Acc:53.74 %  | 	Valid Loss:0.0240 	Valid Acc: 50.37 %


[EPOCH:3]	 Train Loss:0.0177	 Train Acc:62.10 %  | 	Valid Loss:0.0179 	Valid Acc: 61.07 %


[EPOCH:4]	 Train Loss:0.0150	 Train Acc:67.38 %  | 	Valid Loss:0.0160 	Valid Acc: 65.85 %


[EPOCH:5]	 Train Loss:0.0130	 Train Acc:71.92 %  | 	Valid Loss:0.0132 	Valid Acc: 71.48 %


[EPOCH:6]	 Train Loss:0.0113	 Train Acc:75.35 %  | 	Valid Loss:0.0126 	Valid Acc: 73.86 %


[EPOCH:7]	 Train Loss:0.0100	 Train Acc:78.24 %  | 	Valid Loss:0.0137 	Valid Acc: 72.51 %


[EPOCH:8]	 Train Loss:0.0090	 Train Acc:80.61 %  | 	Valid Loss:0.0105 	Valid Acc: 76.48 %


[EPOCH:9]	 Train Loss:0.0079	 Train Acc:83.41 %  | 	Valid Loss:0.0114 	Valid Acc: 75.74 %


[EPOCH:10]	 Train Loss:0.0073	 Train Acc:83.93 %  | 	Valid Loss:0.0099 	Valid Acc: 79.17 %


[EPOCH:11]	 Train Loss:0.0065	 Train Acc:85.73 %  | 	Valid Loss:0.0094 	Valid 

#모델 평가

In [17]:
import os
from tqdm.notebook import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

import torch
from torch import nn, optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import models
from torchvision import transforms as T
from torchvision.datasets import ImageFolder

from torch.backends import cudnn
cudnn.benchmark = True

In [18]:
# 채점시 사용되는 코드
# 본인의 코드에 사용해도 된다.
@torch.no_grad()
def validate_epoch(model: nn.Module, data_loader: DataLoader, device: torch.device):
    ''' data_loader provides inputs and GTs.
        model receives input images from data_loader and produces logits.
        returns the accuracy of model in percent.
    '''
    model.eval()
    accuracies = []
    for images, labels in tqdm(data_loader, total=len(data_loader), mininterval=1, desc='measuring accuracy'):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        pred = torch.argmax(logits, dim=1)

        accuracies.append(pred == labels)

    accuracy = torch.concat(accuracies).float().mean() * 100
    return accuracy.item()

In [19]:
model.to(DEVICE)
validate_epoch(model2, test_loader,device="cuda")

measuring accuracy:   0%|          | 0/28 [00:00<?, ?it/s]

96.03174591064453